In [12]:
from utils.config import *
import utils.df_processor as dfp
import utils.word_segment as ws
import gensim
from IPython.display import display
from sklearn.model_selection import train_test_split

import pandas as pd
import os
saving_dir= 'D:\\CodeRepositories\\py_project\\data_mining\\DataMiningMid_Classification\\data'

### Create Dataframe

In [3]:
df_i = pd.read_csv(I_DATA_DIR, encoding=ENCODING)
df_s = pd.read_csv(S_DATA_DIR, encoding=ENCODING)
df_i.insert(0, 'category', '內科')
df_s.insert(0, 'category', '外科')

In [ ]:
display(df_i['department'].value_counts())
display(df_s['department'].value_counts())

In [6]:
df_all = pd.concat([df_i, df_s], ignore_index=True)

### Remove Stop Words and Word Segment 
- ask: 9m 54.3s
- ans:22m  2.3s

In [9]:
df_all['answer_clean'] = df_all['answer'].apply(lambda x: ws.word_segment(x))
df_all.to_csv(os.path.join(SAVING_DIR, 'df_all_2.csv'), index=False)

In [10]:
display(df_all.head())

,category,department,title,ask,answer,ask_clean,answer_clean
0,內科,心血管科,高血压患者能吃党参吗？,我有高血压这两天女婿来的时候给我拿了些党参泡水喝，您好高血压可以吃党参吗？,高血压病人可以口服党参的。党参有降血脂，降血压的作用，可以彻底消除血液中的垃圾，从而对冠心病...,高血压 两天 女婿 些 党参 泡水 喝 您好 高血压 吃 党参,高血压 病人 口服 党参 党参 降血脂 降血压 作用 彻底消除 血液 中 垃圾 冠心病 心血...
1,內科,心血管科,高血压该治疗什么？,我是一位中学教师，平时身体健康，最近学校组织健康检查，结果发觉我是高血压，去年还没有这种情况...,高血压患者首先要注意控制食盐摄入量，每天不超过六克，注意不要吃太油腻的食物，多吃新鲜的绿色蔬...,一位 中学教师 平时 身体健康 学校 组织 健康检查 发觉 高血压 去年 还 情况 很 担心...,高血压 患者 控制 食盐 摄入量 不 超过 六克 吃 太 油腻 食物 吃 新鲜 绿色 蔬菜水...
2,內科,心血管科,老年人高血压一般如何治疗？,我爷爷今年68了，年纪大了，高血压这些也领着来了，这些病让老人很痛苦，每次都要按时喝药，才能...,你爷爷患高血压，这是老年人常见的心血管病，血管老化硬化，血压调整能力消退了，目前治疗高血压最...,爷爷 年纪 大 高血压 领着 病 老人 很 痛苦 每次 都 按时 喝药 才能 控制 住 还...,爷爷 患 高血压 这是 老年人 常见 心血管病 血管 老化 硬化 血压 调整 能力 消退 治...
3,內科,内分泌科,糖尿病还会进行遗传吗？,糖尿病有隔代遗传吗？我妈是糖尿病，很多年了，也没养好，我现在也是，我妹子也是，我儿子现在二十...,2型糖尿病的隔代遗传概率为父母患糖尿病，临产的发生率为40%，比一般人患糖尿病，疾病，如何更...,糖尿病 隔代遗传 我妈 糖尿病 很多年 没养 好 妹子 儿子 二十岁 没什么 会 糖尿病 ...,型 糖尿病 隔代遗传 概率 父母 患 糖尿病 临产 发生率 人患 糖尿病 疾病 更 选择...
4,內科,内分泌科,糖尿病一般需要怎么治疗？,我妈定期检查仔细检查的时候，仔细检查出患糖尿病，糖尿病需要有怎么治疗？我大概知晓是需要有控制...,糖尿病患者首先通过饮食控制和锻练运动，肥胖患者把体重降下来等方式调整一下看一看，如果血糖仍然...,我妈 定期检查 仔细检查 仔细检查 出患 糖尿病 糖尿病 治疗 大概 知晓 控制 糖量 不...,糖尿病 患者 饮食 控制 锻练 运动 肥胖 患者 体重 降下来 方式 调整 看一看 血糖 偏...


In [16]:
X_train, X_test, Y_train, Y_test= train_test_split(df_all['ask_clean'], df_all['department'], test_size=0.2, shuffle=True)

In [20]:
display(X_train.head())

display(Y_train.head())

228125    儿子 每次 哭闹 肚子 都 会 鼓起来 一个 包包 哭闹 越 厉害 鼓起来 越 厉害 小肠 ...
177822                                            原发性 癫痫 治 
234640    近上 厕所 拉稀 早上 上大厕 便 稀 用力 左上角 腹部 会 疼痛 下 轻按 感觉 请问 ...
289073    患者 年龄  以往 诊断 治疗 效果  作 左 肾 开放式 取石 复发 至今 作 碎石 结石...
253838     工作 原因 两条腿 上 血管 越来越 特别 难受 都 穿 裙子 请问 主动脉 夹层 转院 救治 
Name: ask_clean, dtype: object

228125    普通外科
177822     神经科
234640      肛肠
289073     泌尿科
253838     血管科
Name: department, dtype: object

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
 
tfidf = TfidfVectorizer(analyzer=lambda x: x)
logistic = LogisticRegression(solver='liblinear', multi_class='auto')

tfidf_logistic = Pipeline([
    ('tfidf', tfidf), 
    ('logistic', logistic)
])

In [23]:
from scipy.stats import randint, uniform

# w2v_params = {'w2v__size': [100, 150, 200]}
tfidf_params = {'tfidf__ngram_range': [(1, 1), (1, 2)]}
logistic_params = {'logistic__C': [0.5, 1.0, 1.5]}
# xgb_params = {'xgb__max_depth': randint(low=3, high=12),
#               'xgb__colsample_bytree': uniform(loc=0.8, scale=0.2),
#               'xgb__subsample': uniform(loc=0.8, scale=0.2)}

tfidf_logistic_params = {**tfidf_params, **logistic_params}
# w2v_xgb_params = {**w2v_params, **xgb_params}

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

cv = 3
n_iter = 3
random_state = 1234
scoring = 'accuracy'

all_models = [
    ('tfidf_logistic', tfidf_logistic, tfidf_logistic_params)
]

all_models_info = []
for name, model, params in all_models:
    print('training:', name)
    model_tuned = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        cv=cv,
        n_iter=n_iter,
        n_jobs=-1,
        verbose=1,
        scoring=scoring,
        random_state=random_state,
        return_train_score=False
    ).fit(X_train, Y_train)
    
    Y_test_pred = model_tuned.predict(X_test)
    test_score = accuracy_score(Y_test, Y_test_pred)
    info = name, model_tuned.best_score_, test_score, model_tuned
    all_models_info.append(info)

columns = ['model_name', 'train_score', 'test_score', 'estimator']
results = pd.DataFrame(all_models_info, columns=columns)
results = (results
           .sort_values('test_score', ascending=False)
           .reset_index(drop=True))
results

training: tfidf_logistic
Fitting 3 folds for each of 3 candidates, totalling 9 fits


D:\ProgramFiles\Anoconda\envs\nursing-env\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


,model_name,train_score,test_score,estimator
0,tfidf_logistic,0.686857,0.68881,"RandomizedSearchCV(cv=3,\n e..."


In [ ]:
from xgboost import XGBClassifier

xgb= XGBClassifier(learning_rate= 0.01, n_estimators=100, n_jobs=-1)


In [ ]:
display(df_all.head())

### Saving Text&CSV

In [ ]:
# df_p.i_df.to_csv(os.path.join(saving_dir, 'i_s_ask_ans.csv'), mode='a', columns=['ask'], index=False, header=False)
# df_p.i_df.to_csv(os.path.join(saving_dir, 'i_s_ask_ans.csv'), mode='a', columns=['answer'], index=False, header=False)
# df_p.s_df.to_csv(os.path.join(saving_dir, 'i_s_ask_ans.csv'), mode='a', columns=['ask'], index=False, header=False)
# df_p.s_df.to_csv(os.path.join(saving_dir, 'i_s_ask_ans.csv'), mode='a', columns=['answer'], index=False, header=False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\costco\AppData\Local\Temp\jieba.cache
Loading model cost 0.751 seconds.
Prefix dict has been built successfully.


KeyboardInterrupt: 

In [ ]:
df_p.all_df['answer_clean'] = df_p.all_df['answer'].apply(
    lambda x: ws.word_segment(x))

In [ ]:
display(df_p.all_df.head())

,category,department,title,ask,answer
0,內科,心血管科,高血压患者能吃党参吗？,我有高血压这两天女婿来的时候给我拿了些党参泡水喝，您好高血压可以吃党参吗？,高血压病人可以口服党参的。党参有降血脂，降血压的作用，可以彻底消除血液中的垃圾，从而对冠心病...
1,內科,心血管科,高血压该治疗什么？,我是一位中学教师，平时身体健康，最近学校组织健康检查，结果发觉我是高血压，去年还没有这种情况...,高血压患者首先要注意控制食盐摄入量，每天不超过六克，注意不要吃太油腻的食物，多吃新鲜的绿色蔬...
2,內科,心血管科,老年人高血压一般如何治疗？,我爷爷今年68了，年纪大了，高血压这些也领着来了，这些病让老人很痛苦，每次都要按时喝药，才能...,你爷爷患高血压，这是老年人常见的心血管病，血管老化硬化，血压调整能力消退了，目前治疗高血压最...
3,內科,内分泌科,糖尿病还会进行遗传吗？,糖尿病有隔代遗传吗？我妈是糖尿病，很多年了，也没养好，我现在也是，我妹子也是，我儿子现在二十...,2型糖尿病的隔代遗传概率为父母患糖尿病，临产的发生率为40%，比一般人患糖尿病，疾病，如何更...
4,內科,内分泌科,糖尿病一般需要怎么治疗？,我妈定期检查仔细检查的时候，仔细检查出患糖尿病，糖尿病需要有怎么治疗？我大概知晓是需要有控制...,糖尿病患者首先通过饮食控制和锻练运动，肥胖患者把体重降下来等方式调整一下看一看，如果血糖仍然...


### Main Method